In [ ]:
# --- Κελί 1: Mount + Αντιγραφή από Google Drive ---
from google.colab import drive
import os

drive.mount('/content/drive')

# Δημιουργία φακέλων
!mkdir -p /content/data/new_images

# Αντιγραφή GeoTIFF (.tif) και world files (.tfw) από Drive σε local
!cp /content/drive/MyDrive/road_extraction/new_images/*.tif /content/data/new_images/
!cp /content/drive/MyDrive/road_extraction/new_images/*.tfw /content/data/new_images/



In [ ]:
import numpy as np
import rasterio
from PIL import Image
from tensorflow.keras.models import load_model

# Φόρτωση μοντέλου (προσαρμόστε custom_objects αν χρειάζεται)
model = load_model('/content/drive/MyDrive/road_extraction/road_segmentation_model.h5')

# --- 1. Μεμονωμένη εικόνα ---
def segment_large_geotiff_georeferenced(model, image_path, output_path,
                                        tile_size=256, overlap=0, threshold=0.5):
    with rasterio.open(image_path) as src:
        profile = src.profile
        transform = src.transform
        crs = src.crs
        img_np = src.read()  # (C, H, W)
        img_np = np.transpose(img_np, (1, 2, 0))  # (H, W, C)

    if img_np.shape[2] > 3:
        img_np = img_np[:, :, :3]
    if img_np.ndim == 2:
        img_np = np.stack([img_np]*3, axis=-1)

    h, w, c = img_np.shape
    mask_full = np.zeros((h, w), dtype=np.uint8)
    step = tile_size - overlap

    for y in range(0, h, step):
        for x in range(0, w, step):
            x_end = min(x + tile_size, w)
            y_end = min(y + tile_size, h)
            x_start = x_end - tile_size
            y_start = y_end - tile_size
            if x_start < 0: x_start = 0
            if y_start < 0: y_start = 0

            tile = img_np[y_start:y_start+tile_size, x_start:x_start+tile_size]
            tile_input = tile / 255.0
            tile_input = np.expand_dims(tile_input, axis=0)

            pred = model.predict(tile_input)[0]
            pred_mask = (pred.squeeze() > threshold).astype("uint8") * 255

            mask_full[y_start:y_start+tile_size, x_start:x_start+tile_size] = np.maximum(
                mask_full[y_start:y_start+tile_size, x_start:x_start+tile_size], pred_mask
            )

    profile.update({'count': 1, 'dtype': 'uint8', 'compress': 'lzw'})
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(mask_full, 1)

    return output_path

# --- 2. Επεξεργασία φακέλου ---
def batch_segment_geotiff_folder(model, input_folder, output_folder,
                                  tile_size=256, overlap=0, threshold=0.5):
    os.makedirs(output_folder, exist_ok=True)
    tif_files = sorted([f for f in os.listdir(input_folder) if f.lower().endswith('.tif') and not f.lower().endswith('.tfw')])

    for tif_file in tif_files:
        input_path = os.path.join(input_folder, tif_file)
        output_path = os.path.join(output_folder, f"mask_{tif_file}")

        print(f"▶️ Επεξεργασία: {tif_file}...")
        segment_large_geotiff_georeferenced(model, input_path, output_path,
                                            tile_size=tile_size,
                                            overlap=overlap,
                                            threshold=threshold)
        print(f"✅ Αποθηκεύτηκε: {output_path}")

    print("✅ Ολοκληρώθηκε η επεξεργασία όλων των εικόνων.")

# Εκτέλεση batch
batch_segment_geotiff_folder(
    model=model,
    input_folder="/content/data/new_images",
    output_folder="/content/data/predicted_masks",
    tile_size=256,
    overlap=0,
    threshold=0.5
)


In [ ]:
# --- Κελί 3: Zip + Επιστροφή στο Google Drive ---
!zip -r /content/predicted_masks.zip /content/data/predicted_masks
!cp /content/predicted_masks.zip /content/drive/MyDrive/road_extraction/predicted_masks.zip
